In [1]:
import polars as pl
import os
import json
pl.Config.set_fmt_str_lengths(400)
import requests
from web3 import Web3


In [2]:
DATA_PATH = os.getenv("DATA_PATH", "")

In [3]:
prob_df = pl.read_parquet("/Users/joaovictorassisdasilveira/Desktop/work/bleu/repositories/op-farcaster-social-graph/repo/farcaster-social-graph/farcaster-social-graph-api/farcaster_social_graph_api/data/sybil_scar_results.parquet")
fid_df = pl.read_parquet("/Users/joaovictorassisdasilveira/Desktop/work/bleu/repositories/op-farcaster-social-graph/repo/farcaster-social-graph/notebooks/parquet/full/farcaster-fids-0-1727974800.parquet")
fnames_df = pl.read_parquet("/Users/joaovictorassisdasilveira/Desktop/work/bleu/repositories/op-farcaster-social-graph/repo/farcaster-social-graph/notebooks/parquet/full/farcaster-fnames-0-1727974800.parquet")
verifications_df = pl.read_parquet(f"{DATA_PATH}/farcaster-verifications-0-1727974800.parquet")

In [4]:
print(f"{prob_df.shape[0]/fid_df.shape[0]:.2%}")
prob_df

42.96%


fid_index,posterior,fid
i64,f64,i64
136655,0.0,533141
35041,0.0,331816
276452,0.0,311221
359451,0.0,865104
189230,0.0,681666
…,…,…
135178,1.0,242933
335444,0.0,406441
49010,0.0,301525


In [5]:
prob_df.groupby("posterior").agg(pl.count("posterior").alias("count"))

/var/folders/v9/zdrfs19x765_sy8y01lgc13m0000gn/T/ipykernel_28230/458817810.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  prob_df.groupby("posterior").agg(pl.count("posterior").alias("count"))


posterior,count
f64,u32
0.899764,99
0.405077,3
0.69664,1
0.03792,4
0.973549,1
…,…
0.37671,1
0.944119,1
0.543659,1


In [6]:
print(prob_df.filter(pl.col('posterior')<=0.5).shape[0])
print(prob_df.filter(pl.col('posterior')>0.5).shape[0])
print(f"{(prob_df.filter(pl.col('posterior')>0.5).shape[0]/prob_df.shape[0]):.2%}")

267058
103676
27.97%


In [7]:
bot_of_not_df = pl.read_parquet("/Users/joaovictorassisdasilveira/Desktop/work/bleu/repositories/op-farcaster-social-graph/repo/farcaster-social-graph/farcaster-social-graph-api/farcaster_social_graph_api/persisted_data/bot_or_not_without_info.parquet")

In [8]:
bot_of_not_df

fid,bot
i64,bool
446097,false
3,false
8,false
12,false
2,false
…,…
361320,false
399306,false
21348,false


In [9]:
bots_df = bot_of_not_df.filter(pl.col('bot') == True)
non_bots_df = bot_of_not_df.filter(pl.col('bot') == False)
bots_posterior = prob_df.filter(pl.col('fid').is_in(bots_df.select('fid')))
non_bots_posterior = prob_df.filter(pl.col('fid').is_in(non_bots_df.select('fid')))

In [10]:
print("as sybil (correct)", bots_posterior.filter(pl.col('posterior')<=0.5).shape[0])
print("as benign (wrong)", bots_posterior.filter(pl.col('posterior')>0.5).shape[0])

as sybil (correct) 3424
as benign (wrong) 1211


In [11]:
print("as sybil (wrong)", non_bots_posterior.filter(pl.col('posterior')<=0.5).shape[0])
print("as benign (correct)", non_bots_posterior.filter(pl.col('posterior')>0.5).shape[0])

as sybil (wrong) 2636
as benign (correct) 4528


In [12]:
fnames_df

created_at,updated_at,custody_address,expires_at,fid,deleted_at,fname
datetime[ns],datetime[ns],binary,datetime[ns],i64,datetime[ns],str
2023-09-05 23:23:12.682492,2023-09-15 16:27:36.213,null,null,0,1970-01-20 13:33:05.644,"""mlb.eth"""
2023-09-20 19:54:05.543151,2023-10-06 08:50:22.629,null,null,0,1970-01-20 14:53:59.645,"""spvce.eth"""
2023-09-05 23:20:59.709202,2023-10-11 06:23:00.017,null,null,0,1970-01-20 13:35:49.486,"""tipyourdj.eth"""
2023-09-05 23:24:12.561655,2023-10-11 06:28:08.543,null,null,0,1970-01-20 14:04:35.080,"""talethril.eth"""
2023-09-05 23:20:51.487470,2023-10-19 05:43:17.348,null,null,0,1970-01-20 13:34:33.702,"""mystri.eth"""
…,…,…,…,…,…,…
2024-10-03 16:51:02.859258,2024-10-03 16:51:02.859258,"b""\xcd\xa7W\xfd<\x13\xe8\xe7\xc5\xc5G\x8e\xdc\x0c\xfd\x04&,\x8e\x13""",null,862928,null,"""picklo"""
2024-10-03 16:51:24.773016,2024-10-03 16:51:24.773016,"b""\xd7\xe2(e\x92>+\xb4\x80\x08\xe7\x8d\x01\xad\x9dXJ[\xf9\xc6""",null,862929,null,"""steelninjakarma"""
2024-10-03 16:54:38.716088,2024-10-03 16:54:38.716088,"b""\xfa\xfc\x12\xb8\xf3\xc9\xb5S8\x87$\xc6\x8f\x11\xd1\x8aC,\xaa2""",null,862931,null,"""boopbase"""


In [13]:
bots_posterior.filter(pl.col('posterior')>0.5).sample(1, seed=1) # wrong benign
fid = 202699
fnames_df.filter(pl.col('fid')==fid) # RIGHT INDENTIFICATION OF NON-BOT THAT WAS MARKED AS BOT

created_at,updated_at,custody_address,expires_at,fid,deleted_at,fname
datetime[ns],datetime[ns],binary,datetime[ns],i64,datetime[ns],str
2023-12-09 05:52:19.808906,2024-06-07 18:31:00.055,"b""}1dB\x1ae\x1dK_\xb7r\xf84Y\xda\x03o\xacw\x1d""",null,202699,null,"""roybot"""


In [14]:
print(bots_posterior.filter(pl.col('posterior')>0.5).sample(1, seed=2))
fid = 412661
fnames_df.filter(pl.col('fid')==fid)

shape: (1, 3)
┌───────────┬───────────┬────────┐
│ fid_index ┆ posterior ┆ fid    │
│ ---       ┆ ---       ┆ ---    │
│ i64       ┆ f64       ┆ i64    │
╞═══════════╪═══════════╪════════╡
│ 20287     ┆ 1.0       ┆ 412661 │
└───────────┴───────────┴────────┘


created_at,updated_at,custody_address,expires_at,fid,deleted_at,fname
datetime[ns],datetime[ns],binary,datetime[ns],i64,datetime[ns],str
2024-03-27 19:58:16.869847,2024-06-08 05:25:37.649,"b""\xdf\x0c\x85\xc6\x14\xb1\xcc\xfa\x91'\x14BK\xbd\xcc\xca\x12\xb5`\xbc""",null,412661,null,"""heronimo"""


In [15]:
fid = non_bots_posterior.filter(pl.col('posterior')>0.5).sample(1)[0, "fid"] # right benign
fnames_df.filter(pl.col('fid')==fid)

created_at,updated_at,custody_address,expires_at,fid,deleted_at,fname
datetime[ns],datetime[ns],binary,datetime[ns],i64,datetime[ns],str
2024-01-23 22:05:16.374758,2024-06-07 18:53:00.182,"b""Nq\xceP\xe2\xc5\x17\x16\xabv\xa2Dv'\x94\xa4\xf3J\x0er""",null,231788,null,"""spencecoin"""


In [16]:
fnames_df.filter(pl.col("fid")==860200)

created_at,updated_at,custody_address,expires_at,fid,deleted_at,fname
datetime[ns],datetime[ns],binary,datetime[ns],i64,datetime[ns],str
2024-09-25 15:45:01.137653,2024-09-25 15:45:01.137653,"b""D\xa7\x0aX\x03\xdf\x87\xaa\xfd\x00.\xe3\x15@U*\xb3=\x9bF""",null,860200,null,"""devjoaov"""


In [ ]:
apikey = ""
scorer_id = 1

In [18]:
def get_score(address):
    url = f"https://api.scorer.gitcoin.co/registry/stamps/{address}"
    
    headers = {"X-API-KEY": apikey, "accept": "application/json",}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(response.text)
        response.raise_for_status()

In [19]:
get_score("0xF7EB9267Bac21E05a3d792275ddcBd520B5cacFe")

{'next': None,
 'prev': None,
 'items': [{'version': '1.0.0',
   'credential': {'type': ['VerifiableCredential'],
    'proof': {'type': 'EthereumEip712Signature2021',
     'created': '2024-10-17T20:07:01.329Z',
     '@context': 'https://w3id.org/security/suites/eip712sig-2021/v1',
     'proofValue': '0xb9b5792a415bec647da247a87a2ebc8cbe7f9acb064bc7e0eab720cb10fb8c76420ac16d99dff686d19b47178c2a718432d1b20705a5c27148bd1cd58c3b04021c',
     'eip712Domain': {'types': {'Proof': [{'name': '@context',
         'type': 'string'},
        {'name': 'created', 'type': 'string'},
        {'name': 'proofPurpose', 'type': 'string'},
        {'name': 'type', 'type': 'string'},
        {'name': 'verificationMethod', 'type': 'string'}],
       '@context': [{'name': 'hash', 'type': 'string'},
        {'name': 'provider', 'type': 'string'}],
       'Document': [{'name': '@context', 'type': 'string[]'},
        {'name': 'credentialSubject', 'type': 'CredentialSubject'},
        {'name': 'expirationDate', 

In [20]:
verifications_df

created_at,updated_at,deleted_at,timestamp,fid,hash,claim,id
datetime[ns],datetime[ns],datetime[ns],datetime[ns],i64,binary,str,i64
2023-09-05 23:07:28.395977,2023-09-05 23:07:28.395977,null,2023-07-22 00:08:54,1,"b""S\x02\xf5=\x93\xea\xbf\xb0\xed\xaf\xb9\xce\xf1\xbd\x84\x16\xfd\x8f\x9cF""","""{""address"": ""0x86924c37a93734e8611eb081238928a9d18a63c0"", ""blockHash"": ""0x191905a9201170abb55f4c90a4cc968b44c1b71cdf3db2764b775c93e7e22b29"", ""ethSignature"": ""0x2fc09da1f4dcb7236efb91f77932c249c418c0af00c66ed92ee1f35b02c80d6a1145280c9f361d207d28447f8f7463366840d3a9369036cf6954afd1fd331beb1b""}""",1
2023-09-05 23:07:31.065430,2023-09-05 23:07:31.065430,null,2023-04-28 17:42:20,2,"b""\xa5\x053\x17F\xec\x8cQ\x10\xa9K\xdb\x09\x8c\xd9d\xe4:\x8f+""","""{""address"": ""0x91031dcfdea024b4d51e775486111d2b2a715871"", ""blockHash"": ""0xd74860c4bbf574d5ad60f03a478a30f990e05ac723e138a5c860cdb3095f4296"", ""ethSignature"": ""0xb72c63d61f075b36fb66a9a867b50836cef19d653a3c09005628738677bcb25f25b6b6e6d2e1d69cd725327b3c020deef9e2575a22dc8ed08f88bc75718ce1cb1c""}""",2
2023-09-05 23:07:31.065430,2023-09-05 23:07:31.065430,2024-02-22 22:23:26.823,2023-07-26 20:46:33,2,"b""\x8f\xcc\x9dzl\xf7[\x95\xba\xda\xf0I\xab\xe1\xdc\x9f\x14\xabb(""","""{""address"": ""0x182327170fc284caaa5b1bc3e3878233f529d741"", ""blockHash"": ""0x4470e6a2322145666ea336a35d69ba94a065ce161a825230d6905d62de61ce82"", ""ethSignature"": ""0x21c3036e33104a781f356897623c73a9b3f8b44255d9bf8a81c2f4d9977bb67f057714f2a5be743a322673a22057d817179f6c9b74ca0a32f9b3bdcaf0ef47111b""}""",3
2023-09-05 23:07:32.097405,2023-09-05 23:07:32.097405,null,2023-05-30 18:41:20,5,"b""\xf1\xb8\xc4,\x7f\xeb\x87!\x16\x80q\xd7\xbcY\xb7\x165\x92\xde[""","""{""address"": ""0x4685bb0077179b45095a3c2a175acd9b3cd59d1c"", ""blockHash"": ""0xbd8b7b47d6220d3503407bb4867701206c128952a459a25a33d07bef10d86ed0"", ""ethSignature"": ""0x8bec961e641ab786ea7137a5959c2fe9c004c82970080a022214a29a8347c8e7391c02c7c07dc2e99da79140fd8980caee29d1bdf50629c55c19ce67c53946f81c""}""",4
2023-09-05 23:07:32.371010,2023-09-05 23:07:32.371010,2024-01-30 02:56:41.505,2023-06-15 20:08:07,6,"b""i\xfcX\xd1\xf8\xa3\x1b\x85\xa4\x0cT\x16T\x89/\xe7\xa436\xc1""","""{""address"": ""0x888b4bee0f8dff4e7269811e066a1de6438cbf7a"", ""blockHash"": ""0x9b56b788d267e40627e797a31e044c7b26eb779f27018d52c5bdb053690abdd6"", ""ethSignature"": ""0xcd5e06f1191270429c7a007a4915b217e30048c955503df478a90a2ace8854bf2fb4a1db42c6f90861b9236b3b9efc10660811929fc55750a98c2bb027df82071c""}""",5
…,…,…,…,…,…,…,…
2024-10-03 16:54:12.455801,2024-10-03 16:54:12.455801,null,2024-10-03 16:54:12,859362,"b""\x17@\x9a\xb2\xc4\x94\xb4\x82\xb6\xf1\xcc\xa3\xc7\x99\xe5\x1a\x1e\x92\x8ei""","""{""address"": ""0x6022c231df4f4a030ec8ef5480f3a0c5dd5d10da"", ""blockHash"": ""0xdf4b4bd1e2511e7c9ff720b0f3d0e2d495c88785ef91f7a3094cc08a12ff5d26"", ""ethSignature"": ""0xc4cb3f58e04b48872d7114fbde1ecd4fd0534e20dc0f5e28503f9199f1aceeda39a7f34892b9ff36b217403b24f04741d798f78a8bca3e2dab97ed58155799211c""}""",36445463
2024-04-10 20:15:10.891345,2024-10-03 16:54:43.469,2024-10-03 16:54:43,2024-04-10 07:13:04,470874,"b""\xf7Fx6\xfc\x91\x037\x8c\xe4\x88\x0b\xaa\x81\x88\xbb]\x93\xcd\xe7""","""{""address"": ""0xa8e645bfa0ea8d7836ae7cdd01f81cd5a154ef5a"", ""blockHash"": ""0xd2ea8cf89017e709a2ccb1fc1622632557e7a2df57b7cbf4a1130980cf0aa470"", ""ethSignature"": ""0x977ae76c9d5ee8a081e84dd8fb7997a4e1b23d043f6c8109d15131272ef498c24faa2e5c9043c50d85ed2707288c793369418d0eb28e987789b1744596676c801b""}""",22199884
2024-10-03 16:55:06.423099,2024-10-03 16:55:06.423099,null,2024-10-03 16:55:06,470874,"b""I\x87c\x05\xc1\xf5m\x8b4Y<r\x099\x1bu\xb9\xd0Fi""","""{""address"": ""0xd8da846a59e5577f7ad6cf43fb24b2679f1c8926"", ""blockHash"": ""0x1c31f998acecebad4cf02ddc5ab94dd50707e97c117ec9333220a8f4a4d3e2cf"", ""ethSignature"": ""0x6bd7a963d7075d315281719e4d71626106b27009f5edfc40d35f373ace5af0fb3f7a6c922dc8a1bafa2afb818dbceaa93a76c61e86a6ddfa5645f098270e23711c""}""",36445464


In [21]:
verifications_df.filter(pl.col("claim").str.contains("0x86924c37a93734e8611eb081238928a9d18a63c0"))

created_at,updated_at,deleted_at,timestamp,fid,hash,claim,id
datetime[ns],datetime[ns],datetime[ns],datetime[ns],i64,binary,str,i64
2023-09-05 23:07:28.395977,2023-09-05 23:07:28.395977,null,2023-07-22 00:08:54,1,"b""S\x02\xf5=\x93\xea\xbf\xb0\xed\xaf\xb9\xce\xf1\xbd\x84\x16\xfd\x8f\x9cF""","""{""address"": ""0x86924c37a93734e8611eb081238928a9d18a63c0"", ""blockHash"": ""0x191905a9201170abb55f4c90a4cc968b44c1b71cdf3db2764b775c93e7e22b29"", ""ethSignature"": ""0x2fc09da1f4dcb7236efb91f77932c249c418c0af00c66ed92ee1f35b02c80d6a1145280c9f361d207d28447f8f7463366840d3a9369036cf6954afd1fd331beb1b""}""",1


In [22]:
fid = 3
print(bot_of_not_df.filter(pl.col("fid")==fid))
print(prob_df.filter(pl.col("fid")==fid))

shape: (1, 2)
┌─────┬───────┐
│ fid ┆ bot   │
│ --- ┆ ---   │
│ i64 ┆ bool  │
╞═════╪═══════╡
│ 3   ┆ false │
└─────┴───────┘
shape: (1, 3)
┌───────────┬───────────┬─────┐
│ fid_index ┆ posterior ┆ fid │
│ ---       ┆ ---       ┆ --- │
│ i64       ┆ f64       ┆ i64 │
╞═══════════╪═══════════╪═════╡
│ 27698     ┆ 1.0       ┆ 3   │
└───────────┴───────────┴─────┘


In [23]:
print("as sybil (wrong)", non_bots_posterior.filter(pl.col('posterior')<=0.5).shape[0])


as sybil (wrong) 2636


In [24]:
wrong_fids = non_bots_posterior.filter(pl.col('posterior')<=0.5).sample(10, seed=10).select("fid")

In [25]:
verifications_df.filter(pl.col('fid').is_in(wrong_fids))

created_at,updated_at,deleted_at,timestamp,fid,hash,claim,id
datetime[ns],datetime[ns],datetime[ns],datetime[ns],i64,binary,str,i64
2023-09-05 23:21:14.783932,2023-09-05 23:21:14.783932,null,2023-08-31 08:56:28,14566,"b""i\xa6\xcd\xfe\xa2\xee}\xfa\x9b\xb5f\xc6:0TG.T\xb3\x85""","""{""address"": ""0x26ef3689bf05e05ab15728fbc7863a12eb62d2fd"", ""blockHash"": ""0x3e2b0af842a4890599700182b3f7b0514d49777c13b0787540d81af048ae7b07"", ""ethSignature"": ""0x1fc4a6928c8ccbc253ae25b4f4357d1dd04d5d52f9926a688dab380b593e291401b3be22a7f23876b60ebf776d3a4165bf6d00406d938041d8c8ea8936c67fe11c""}""",5762
2024-02-03 18:11:53.927103,2024-02-03 18:11:53.927103,2024-02-03 18:13:22.956,2024-02-03 18:11:48,267721,"b""\xf0\xbf{\x1bX+J\xffH=n\xd1\x93\x94\xaf\x1b\x191\xd0\x16""","""{""address"": ""0x4509deb8ecac34003463c745f5df0e6931a86f3a"", ""blockHash"": ""0xb82eb10bd69401d64e5eed7e93ff8a282a5fb8a546c34574bea9d41df1bbc518"", ""ethSignature"": ""0xbe9779b8aa15593f50cdcab779f27fabe8af1888317391a62a2e27ee2fb99d0f0062103a592925f5db1832bd3ec27f2e441475b1cd196f84bae4429df75340441c""}""",867348
2024-02-03 18:13:22.950190,2024-02-03 18:13:22.950190,2024-02-03 18:13:38.316,2024-02-03 18:13:16,267721,"b""g\xd7\xa2\x00!\xba\xd2\xe3~\xd3\x09\xe4\x9d\xb5!\x9b\xf1\xfaIi""","""{""address"": ""0x4509deb8ecac34003463c745f5df0e6931a86f3a"", ""blockHash"": ""0x0554b3d6df83d1102ef19ada5534df74061d06ac30c2eca9166aa5b8ee1cb910"", ""ethSignature"": ""0xa2014c50e4433591c095c0b3a2d1741ac28760ff47c1f87c7152abdda87950d555081f18a158c061c1baa700fcdc0e8c4049b72a55dcb97967dc0db543a405191c""}""",867366
2024-02-03 18:13:38.312727,2024-02-03 18:13:38.312727,null,2024-02-03 18:13:32,267721,"b""\xd1\x0e/\xb0u\x14\xb5\xc8K\xe5T\x8a\x00\x8c\xf4{q\xadl\xd9""","""{""address"": ""0x4509deb8ecac34003463c745f5df0e6931a86f3a"", ""blockHash"": ""0xe17ee855b4e46bf9d6fb54cd8bf3a700168181be43685bc5336a895e85db3b66"", ""ethSignature"": ""0xe8b7ac3fea5c65869cc1a0f94ff40892ca3ecd5d06f1e0e99ceb2dc83586006e69f903b226d1e0ce70c3e597a2febcf69f794c49889b0da6e54568799713ed3f1b""}""",867369
2024-02-04 00:12:52.602574,2024-02-04 00:12:52.602574,2024-02-08 13:47:56.158,2024-02-03 15:07:10,246047,"b""\x06\x16\xc1\xe1\xa9\x86O\x10\x02\xb1\xf5')\xc6\x9f(s\x93\xb4\xe9""","""{""address"": ""0x96c04d7fe2981ee19c0a2e74b52247b726572310"", ""blockHash"": ""0xd6d316e647b82cabd14c08e6c802d3ccd0400fcb16d9a70ff79a044735c3760a"", ""ethSignature"": ""0x3df5087f09f63f99d198147477b5adfb06debb5f97488665acc6b5d9a0131f8b16d4b6c2a0a683ea58f22fc252d67fbc72c937ee7cfd9b73899551ed5899ee401c""}""",929628
…,…,…,…,…,…,…,…
2024-06-10 07:20:38.503624,2024-06-12 06:14:45.141,2024-06-12 06:14:44,2024-06-10 07:20:38,483110,"b""fr\xa2\xa9\xde\x1d\x09/F\x1b\xc3\x12\xa0\xdc\xc6fk\x9b{\x0f""","""{""address"": ""0x598202593e149d458a95b91b357cc7ec37043e88"", ""blockHash"": ""0x4b0dcadc9c99f56f181e8f4090ede210f45521cd7fa47557a1270306e8bef54e"", ""ethSignature"": ""0xac2facd5a5c8ed0cc12cbe17ebb57c782e69050b5b7bf22b33b73a870d6e71e7683516df8f6751d17ff65aaad9e9aab6ad3302623613e727c1a734f07926c6b61b""}""",36251168
2024-06-12 06:17:17.874133,2024-06-12 06:17:17.874133,null,2024-06-12 06:17:13,483110,"b""\xe8\xa3\x90G@\x0e\x8a\xdd\xa8\x9en\x9b4\x95,_\xd0\xff\xbc+""","""{""address"": ""0x86e649096b5c7a945c3e6fe2614d1985b48fd2ec"", ""blockHash"": ""0xbb541c08389253939eac39409fbd309e10252760a4f655693f0acf5c5a80541f"", ""ethSignature"": ""0xa38472416730ee62be5e7be9696d67e9d66ed72829f294114eac954e28f6e7e406ad0c753bf496e14fc030f06a3e32b08a82726c462c6a110130b7e7dca7a8e11b""}""",36258115
2024-07-08 08:33:03.471244,2024-07-08 08:33:03.471244,null,2024-07-08 08:33:02,663773,"b""X\x09\x84wm\xe2\xee{o\xe2\x08\x95\x86\x0c\xeb\xf4\x8c\xe2q?""","""{""address"": ""0xf276eed3aa8eb37fc4cbca94e758cdc017b94433"", ""blockHash"": ""0x359f0b78ea039eb3e390c9c3f79e75ae4f1dea47ace950e796dbc90366889315"", ""ethSignature"": ""0x6263cb4c2a4f837c6cf811f4a681bbd33b0dcd129cc5b238caa8446c173c69a02a9766710082fd5a386c2d342acc696a13ebffcf757105995f0cb5a339f19ce71c""}""",36345357


In [26]:
fnames_df.filter(pl.col('fid').is_in(wrong_fids))

created_at,updated_at,custody_address,expires_at,fid,deleted_at,fname
datetime[ns],datetime[ns],binary,datetime[ns],i64,datetime[ns],str
2024-05-24 14:21:36.171700,2024-06-07 15:40:29.413,"b""\xc6\x18G\xbe\x1a\xc2\xdd\x0di\xe5\x9d\xa3v3P\xe9\xa9\xfa\x18s""",null,569928,null,"""kirill4real"""
2024-02-06 12:26:42.310601,2024-06-07 17:28:49.482,"b""\xe0I@\xe3P\xe6\xd3\xae\x1f\xa4W\x87\x03n\xd6^eq.\x0a""",null,14566,null,"""tiger7"""
2023-09-17 13:03:12.976644,2024-06-07 17:28:50.365,"b""&\xef6\x89\xbf\x05\xe0Z\xb1W(\xfb\xc7\x86:\x12\xebb\xd2\xfd""",null,14566,null,"""ttcompany.eth"""
2024-01-31 06:28:57.480636,2024-06-07 19:23:52.132,"b""\x01\xde\x86\xa2\xe3\xcbuW\xa0k""\x1f\xe8\xd4\x7f\x1c\xc5\xb1|l""",null,246047,null,"""dyrano"""
2024-04-04 14:43:37.795152,2024-06-07 19:23:52.166,"b""\x96\xc0M\x7f\xe2\x98\x1e\xe1\x9c\x0a.t\xb5""G\xb7&W#\x10""",null,246047,null,"""dogncat.eth"""
…,…,…,…,…,…,…
2024-02-09 19:23:42.646633,2024-06-08 00:18:05.506,"b""\x15\xa0\x070\xcbr\xa5\xdb\xaa'|\xf5\xdc\xf9V%\x1a\x88\xc6\xe1""",null,331344,null,"""aliparsa"""
2024-03-27 19:41:05.392183,2024-06-08 05:25:32.393,"b""Iqi\x1a\x85\xb8\x97\xfe\xa1\x87X\xfb\x01\xea\x06\x8bH\xc5\xde\x87""",null,412620,null,"""farzadrahnama"""
2024-03-29 20:21:18.864038,2024-06-08 05:55:51.449,"b""\xd24\xa5\xda\xc9\xea\xac\x92[0\xd1I\xd1\xee\x98""\xc8\x1d\xd2\x80""",null,417626,null,"""jagermeister"""


In [27]:
scarbenign_bots = bots_posterior.filter(pl.col('posterior')>0.5)
scarbenign_bots

In [28]:
verifications_df.filter(pl.col('fid').is_in(scarbenign_bots.select('fid')) & pl.col('deleted_at').is_not_null())

created_at,updated_at,deleted_at,timestamp,fid,hash,claim,id
datetime[ns],datetime[ns],datetime[ns],datetime[ns],i64,binary,str,i64
2023-09-05 23:10:23.469279,2023-09-05 23:10:23.469279,2023-09-05 23:54:18.257,2023-06-27 20:45:20,1731,"b""\xfc:^\x95\x02\xf1?\xa4\xa9\xd3\x1f\x17\xd8P\x19?\x83Q\xc5\xb2""","""{""address"": ""0x4cbc2e1c86c5ea7c5ea742a4e77696e4852840b5"", ""blockHash"": ""0x6196251704248b17207027ee9adce479231ef97865396ce629efbca62935c939"", ""ethSignature"": ""0xa59fcff869dc10bc6465ee34a60b08df1e454bf1d12650a464f7ebd231db35724135ec5b0bec4e2151b478414150a36c31846905cf3acec9a389e87301a92d751c""}""",699
2023-09-05 23:10:50.449036,2023-09-05 23:10:50.449036,2023-11-22 00:03:52.450,2023-06-23 23:18:39,2183,"b""\x8e\x8a__P\x13\xa2\x9b$\xb7\xe8_f'7\xd3j\xbc\xb7\xf5""","""{""address"": ""0xb4fc9a7b8be45c14b6a7364667069544ea4cbde0"", ""blockHash"": ""0xe54b64bb3d49488b375e423ea1b649ec1e6baff7eab5dcb36b14b0f29f34df95"", ""ethSignature"": ""0xf04a92b324554ddedde230d85cc2e60d6826ccc6754442e4d7e47111e90405c5508b87ff9d06146b884cf0bcdf19fbf6bd06b6657130fed5f38c4bc03ba0c8e71c""}""",829
2023-09-05 23:10:57.216645,2023-09-05 23:10:57.216645,2024-02-08 13:31:21,2023-06-13 22:57:21,2283,"b""\xc3\x96\xda\x12\x061\xd7~\xb8\x06^\xcbU\x00\xf4\xd1|\xbc""\xdf""","""{""address"": ""0xc5d52fff41386bf8ec591d97441b89053e222e74"", ""blockHash"": ""0x39597ceee9a2b8aec64916b9af8dd5bba052702ded13a1fb5bfb2102e74927b6"", ""ethSignature"": ""0xce34964bcfe2b9a730ccb0d2e24f80cf1def314c33e431e4111709cc8e3df26e4cbe649dfce81315903add89539b747ccb2b05c34854ccf224c2b88ae167892f1c""}""",875
2023-09-05 23:15:24.881911,2023-09-05 23:15:24.881911,2023-12-08 20:58:40.189,2023-06-15 08:05:21,7438,"b""\xe8\xf7\x12\x12\xfd\xfeR,\x99>\x147\x20\x90M\xbf\xb5\xddS'""","""{""address"": ""0x09de8de37fd4befe77a45eda7e794a2bebbdc1b7"", ""blockHash"": ""0x023176771f5b1524335be8370ae1be804b2e1285e52da2bac32ac18242e994da"", ""ethSignature"": ""0x3252adc354c416b73c5d46f3b930edc3eb139832d3125182f9b7941691f67e6566109dd86502f31b43ee0b1e50252a47d354c9a6a2d99285a9038a03b3de666b1c""}""",2341
2023-09-05 23:17:03.049898,2023-09-05 23:17:03.049898,2023-11-17 03:41:47.762,2023-06-28 22:13:27,9595,"b""\xa642E\x04\xe0\xe7\xc1\x0d\xe7\xf8q!\xb9i\xdfT\x84C:""","""{""address"": ""0x81947aadf64cd22ce96999e4c76efb0765839bef"", ""blockHash"": ""0x481aa3e54fb1cb28d68d8bfccce1189e82217f8c7daab49cb106669f0b4245e4"", ""ethSignature"": ""0x29c9da700a25f30254bcc3c7902e39eb888c8f98d495f6f37bbe0393ed3627e6713e861fcbf764b592212229df2b5809054ca652a3c2a918d91c8468e9c7ed391c""}""",2993
…,…,…,…,…,…,…,…
2024-02-04 18:59:29.407448,2024-10-01 21:30:19.169,2024-10-01 21:30:18,2024-02-04 12:48:05,243906,"b""\x85\x0e\x05\x8e(\xcb\\xfa\x8bT\xefKU/\xa0\xae-\xde\xecP""","""{""address"": ""0xd2e34a23e385a4602abfd7408e35375a1d93c121"", ""blockHash"": ""0x2f3308fed68bc25df298524b0e23de1876192695fe545bd4e9ac0800f5cedf4a"", ""ethSignature"": ""0x9ab0b2ba931c14ed4088c00ecd1eea2415fae8d0f794260822a66f0dd7495cd606bccb174ed9fbe8494ca78e4cab9cb50f6cea9777bb7d18e2e8c35784874bd81b""}""",1017499
2024-02-04 22:44:01.703398,2024-10-01 21:30:19.169,2024-10-01 21:30:18,2024-02-04 22:44:00,243906,"b""3\x10QUo\xce\xf7_jU\xe0\x1b\xb94\xe3\x05\xa6sd\x82""","""{""address"": ""0xd2e34a23e385a4602abfd7408e35375a1d93c121"", ""blockHash"": ""0xc234b2e9c0d9f7f7d718a39632ac1cdfce5f892fd55f750eec37f0b42436c507"", ""ethSignature"": ""0x6b65091de64718b0c21f0f261ffd43c7d9fd4b7f35b1b6eef30eb347ea7c84181210f0cf1f3d8d80362e2cfd0749c9288d19ec3ee8fec636e71467c47479c0a41c""}""",1061100
2024-03-27 13:34:55.046858,2024-10-01 21:30:19.169,2024-10-01 21:30:18,2024-03-27 13:34:55,243906,"b""\x0dP\x14\xe1\x98\x14\xeaR\xefd9\x0d\x15~|M\x8f\xfc\x0d\xbe""","""{""address"": ""0xd2e34a23e385a4602abfd7408e35375a1d93c121"", ""blockHash"": ""0x1222e77609726d5f3e5fb70e30f9afb8090e6cf58b44bc3de26b3acde657bccc"", ""ethSignature"": ""0x48fd21db198afcaf4d09578e9accd5dab12adceaec2008a28bd1098d018c71a1536d1854b658c9637087c17a11a9e3443f1960edc3bec002e88805bad7b1e37e1b""}""",15

In [30]:
scarbots_bots = bots_posterior.filter(pl.col('posterior')<=0.5)

In [31]:
verifications_df.filter(pl.col('fid').is_in(scarbots_bots.select('fid')) & pl.col('deleted_at').is_null())

created_at,updated_at,deleted_at,timestamp,fid,hash,claim,id
datetime[ns],datetime[ns],datetime[ns],datetime[ns],i64,binary,str,i64
2023-09-05 23:15:34.619278,2023-09-05 23:15:34.619278,null,2023-03-21 16:44:46,7681,"b""\x82mQ\xc6kK\x1ev8\xab\xbd\x92\xee\x12\xa5B\x0fJO7""","""{""address"": ""0xc2e6b265cb965ded721566f0f9eb5ab1a6162a21"", ""blockHash"": ""0xfe1a21dca8baf26d780c4b8783e48fc41e12cb791bfea3ac819c758bec994cf4"", ""ethSignature"": ""0x33c9be007389b390c17e0767db759742fadd296d5e598e287df155ef814ba346099024ee379d57fb96318973df6fe6cee224f8c78c29636bc2ac616028695e1b1c""}""",2407
2023-09-05 23:20:39.845848,2023-09-05 23:20:39.845848,null,2023-06-24 10:00:14,13991,"b"".\xce6\xd0\xd9\x9eS\xae2\xfd\x1dZ\x16\x8b\x7f_\x9f\x09\xe8y""","""{""address"": ""0x0704bd396d425f2e794e9b921bff16c020e4a95f"", ""blockHash"": ""0xb3a074851aac0ce1f6a3493a3e1b6930032586527b7d01b8177d3b8839f1b21f"", ""ethSignature"": ""0x9d6da235e529191117d1269b0dd50502ace70173776c44abeb230269019989bc5b0e325293606dac08a9ab066d9b3b614e3a1ab8240c27808d0d79598bba62421b""}""",5428
2023-09-05 23:20:54.055477,2023-09-05 23:20:54.055477,null,2023-06-29 21:07:04,14272,"b""\xaa\xb4\xe9\x10\xc2*\xb0\xd0f\xf1\xcfe\x05\x92\xdb\xbf\x0f(!H""","""{""address"": ""0xaaefece1eb73fa752890f6bc63810b5b6e2b81b6"", ""blockHash"": ""0x1f6548a461fb77a5e521eeaad95c2d9a1ef0004344a24dcec5bb6c088a3c21ac"", ""ethSignature"": ""0x4c8c8e91670e7fca4d3330423c5fdcfa49914d0b6225ad934e4d39de8b3904d748c1c2a4578b97b77ae53650faf3d529eae685cedeea6f807674cb99a96dc8f41b""}""",5576
2023-09-05 23:20:56.076400,2023-09-05 23:20:56.076400,null,2023-09-03 12:15:15,14300,"b""'\xa5:\xf8O\x91\xa6\xdd'@\xdfn\xe1\xa4R%\xac\xcd\x1b\xb5""","""{""address"": ""0x75c692dffea2ccaed41291ce39911a0bfe8fde75"", ""blockHash"": ""0x223526d420cc646b70e15f9a8f2ae0cf9759850b51801801ae8a11def9739d8b"", ""ethSignature"": ""0x18f37a34935ba62d3c17506641ee67cadac9aec60838b7b7c04ff3344d936f2843ca3d9311548da5228c568b4f5e471e78760125df4d62c6b7f4bf5b7bf1d4661b""}""",5590
2023-09-05 23:21:00.435773,2023-09-05 23:21:00.435773,null,2023-07-01 20:38:17,14363,"b""K\xa0\xe2\xb0\xc7z\x87p]\x12\xe4\xbfG\xe6(k\xcd\xb1\xc45""","""{""address"": ""0x15c428d154d71558cf18446058428f9788aecbd2"", ""blockHash"": ""0xb7cb56f044522d2615e4bf37257a150ee8405ac513047d668d16ca39311c1fb4"", ""ethSignature"": ""0x901587e3736a969586100e7a80cee0996338ba815fb5e157a2f565a53cc9c0151e604aa9c5b4c6ef040495211a6540def430faedfc03752e913d53a85729c7e61c""}""",5629
…,…,…,…,…,…,…,…
2024-10-01 13:31:28.682968,2024-10-01 13:31:28.682968,null,2024-10-01 13:31:28,323631,"b""\xb3}f\xa0\xd9\x96M\xb1\x7f\x03\xe6-U\xc1\xb5\x15R\xbd\xbe\x84""","""{""address"": ""0x84255c4c1521af4958b7fa35a92264b498c3ee21"", ""blockHash"": ""0x4c5150732e5b2fcb6a0a7afc309e5314124fe582d8e1bac45232c2144e92628c"", ""ethSignature"": ""0xf87ae839e7987a06207cd2239fc17f43d50155e3c03a553892f955ed65142fe964fd4474f25dd68ad9884b86d82bdb3ed585706fdfaf405170b4c251b7a317a41c""}""",36444772
2024-10-01 13:36:35.695883,2024-10-01 13:36:35.695883,null,2024-10-01 13:36:35,323631,"b""\xddO\x9f[\xcf#\xb7\xf0\x86\xc5k\xdeep\x91\xb8h\xabG\xbd""","""{""address"": ""0x84255c4c1521af4958b7fa35a92264b498c3ee21"", ""blockHash"": ""0xc4e4cf43ac8f5b8bb5e50b50642cfea0331fde341d21354a40ec8b265b8f2fba"", ""ethSignature"": ""0x984ffcb088f5e420e637f8bf9182b4b396178872d5693c3ea6c06aae6037a7dc59f558e8e832d6c663ed7a63a25a33576c864bc9f9f61d8369ef5315909bb82a1c""}""",36444775
2024-10-03 01:18:31.847308,2024-10-03 01:18:31.847308,null,2024-10-03 01:18:31,361844,"b""\x88\x83\x16\xf5\x07\x88\x0b\x0c_p\xa8\xa2\xcf}\xfe}\x07S\xbd\x90""","""{""address"": ""0xd0667b51a4185a6fd7617aa858c710ce30649775"", ""blockHash"": ""0xb5345c14acef68acc04215d005f84d251525a99ed240c55b0603dc5d3c73d112"", ""ethSignature"": ""0x6fc763b90cb953730d138bc3dc4ba258154cbdab339006a61ae43be1ec1ab3a51339513a54c37928c0b9fbe160911a6976ce28f6b4d5ed08d0e52e70bb47ac921c""}""",36445274


In [32]:
fnames_df.filter(pl.col('fid')==361844)

created_at,updated_at,custody_address,expires_at,fid,deleted_at,fname
datetime[ns],datetime[ns],binary,datetime[ns],i64,datetime[ns],str
2024-02-19 19:07:20.031709,2024-06-08 01:33:37.140,"b""_\x06=""\xda\xd2\xa0K\x1a+;\xfd\x01,\x03\xe7W\xff\x99E""",null,361844,null,"""timasidra"""


In [38]:
sybilbenign_bots_verifications = verifications_df.filter(pl.col('fid').is_in(scarbenign_bots.select('fid')) & pl.col('deleted_at').is_not_null())
sybilbenign_bots_verifications

created_at,updated_at,deleted_at,timestamp,fid,hash,claim,id
datetime[ns],datetime[ns],datetime[ns],datetime[ns],i64,binary,str,i64
2023-09-05 23:10:23.469279,2023-09-05 23:10:23.469279,2023-09-05 23:54:18.257,2023-06-27 20:45:20,1731,"b""\xfc:^\x95\x02\xf1?\xa4\xa9\xd3\x1f\x17\xd8P\x19?\x83Q\xc5\xb2""","""{""address"": ""0x4cbc2e1c86c5ea7c5ea742a4e77696e4852840b5"", ""blockHash"": ""0x6196251704248b17207027ee9adce479231ef97865396ce629efbca62935c939"", ""ethSignature"": ""0xa59fcff869dc10bc6465ee34a60b08df1e454bf1d12650a464f7ebd231db35724135ec5b0bec4e2151b478414150a36c31846905cf3acec9a389e87301a92d751c""}""",699
2023-09-05 23:10:50.449036,2023-09-05 23:10:50.449036,2023-11-22 00:03:52.450,2023-06-23 23:18:39,2183,"b""\x8e\x8a__P\x13\xa2\x9b$\xb7\xe8_f'7\xd3j\xbc\xb7\xf5""","""{""address"": ""0xb4fc9a7b8be45c14b6a7364667069544ea4cbde0"", ""blockHash"": ""0xe54b64bb3d49488b375e423ea1b649ec1e6baff7eab5dcb36b14b0f29f34df95"", ""ethSignature"": ""0xf04a92b324554ddedde230d85cc2e60d6826ccc6754442e4d7e47111e90405c5508b87ff9d06146b884cf0bcdf19fbf6bd06b6657130fed5f38c4bc03ba0c8e71c""}""",829
2023-09-05 23:10:57.216645,2023-09-05 23:10:57.216645,2024-02-08 13:31:21,2023-06-13 22:57:21,2283,"b""\xc3\x96\xda\x12\x061\xd7~\xb8\x06^\xcbU\x00\xf4\xd1|\xbc""\xdf""","""{""address"": ""0xc5d52fff41386bf8ec591d97441b89053e222e74"", ""blockHash"": ""0x39597ceee9a2b8aec64916b9af8dd5bba052702ded13a1fb5bfb2102e74927b6"", ""ethSignature"": ""0xce34964bcfe2b9a730ccb0d2e24f80cf1def314c33e431e4111709cc8e3df26e4cbe649dfce81315903add89539b747ccb2b05c34854ccf224c2b88ae167892f1c""}""",875
2023-09-05 23:15:24.881911,2023-09-05 23:15:24.881911,2023-12-08 20:58:40.189,2023-06-15 08:05:21,7438,"b""\xe8\xf7\x12\x12\xfd\xfeR,\x99>\x147\x20\x90M\xbf\xb5\xddS'""","""{""address"": ""0x09de8de37fd4befe77a45eda7e794a2bebbdc1b7"", ""blockHash"": ""0x023176771f5b1524335be8370ae1be804b2e1285e52da2bac32ac18242e994da"", ""ethSignature"": ""0x3252adc354c416b73c5d46f3b930edc3eb139832d3125182f9b7941691f67e6566109dd86502f31b43ee0b1e50252a47d354c9a6a2d99285a9038a03b3de666b1c""}""",2341
2023-09-05 23:17:03.049898,2023-09-05 23:17:03.049898,2023-11-17 03:41:47.762,2023-06-28 22:13:27,9595,"b""\xa642E\x04\xe0\xe7\xc1\x0d\xe7\xf8q!\xb9i\xdfT\x84C:""","""{""address"": ""0x81947aadf64cd22ce96999e4c76efb0765839bef"", ""blockHash"": ""0x481aa3e54fb1cb28d68d8bfccce1189e82217f8c7daab49cb106669f0b4245e4"", ""ethSignature"": ""0x29c9da700a25f30254bcc3c7902e39eb888c8f98d495f6f37bbe0393ed3627e6713e861fcbf764b592212229df2b5809054ca652a3c2a918d91c8468e9c7ed391c""}""",2993
…,…,…,…,…,…,…,…
2024-02-04 18:59:29.407448,2024-10-01 21:30:19.169,2024-10-01 21:30:18,2024-02-04 12:48:05,243906,"b""\x85\x0e\x05\x8e(\xcb\\xfa\x8bT\xefKU/\xa0\xae-\xde\xecP""","""{""address"": ""0xd2e34a23e385a4602abfd7408e35375a1d93c121"", ""blockHash"": ""0x2f3308fed68bc25df298524b0e23de1876192695fe545bd4e9ac0800f5cedf4a"", ""ethSignature"": ""0x9ab0b2ba931c14ed4088c00ecd1eea2415fae8d0f794260822a66f0dd7495cd606bccb174ed9fbe8494ca78e4cab9cb50f6cea9777bb7d18e2e8c35784874bd81b""}""",1017499
2024-02-04 22:44:01.703398,2024-10-01 21:30:19.169,2024-10-01 21:30:18,2024-02-04 22:44:00,243906,"b""3\x10QUo\xce\xf7_jU\xe0\x1b\xb94\xe3\x05\xa6sd\x82""","""{""address"": ""0xd2e34a23e385a4602abfd7408e35375a1d93c121"", ""blockHash"": ""0xc234b2e9c0d9f7f7d718a39632ac1cdfce5f892fd55f750eec37f0b42436c507"", ""ethSignature"": ""0x6b65091de64718b0c21f0f261ffd43c7d9fd4b7f35b1b6eef30eb347ea7c84181210f0cf1f3d8d80362e2cfd0749c9288d19ec3ee8fec636e71467c47479c0a41c""}""",1061100
2024-03-27 13:34:55.046858,2024-10-01 21:30:19.169,2024-10-01 21:30:18,2024-03-27 13:34:55,243906,"b""\x0dP\x14\xe1\x98\x14\xeaR\xefd9\x0d\x15~|M\x8f\xfc\x0d\xbe""","""{""address"": ""0xd2e34a23e385a4602abfd7408e35375a1d93c121"", ""blockHash"": ""0x1222e77609726d5f3e5fb70e30f9afb8090e6cf58b44bc3de26b3acde657bccc"", ""ethSignature"": ""0x48fd21db198afcaf4d09578e9accd5dab12adceaec2008a28bd1098d018c71a1536d1854b658c9637087c17a11a9e3443f1960edc3bec002e88805bad7b1e37e1b""}""",15

In [ ]:
verifications_dict = {}

In [47]:
json.loads(sybilbenign_bots_verifications.select("claim")[0])

TypeError: the JSON object must be str, bytes or bytearray, not DataFrame

In [53]:
json.loads(sybilbenign_bots_verifications[0, "claim"])["address"]

'0x4cbc2e1c86c5ea7c5ea742a4e77696e4852840b5'

In [61]:
verifications_dict = {}
for row in sybilbenign_bots_verifications.select(["fid", "claim"]).to_dicts():
    address = json.loads(row["claim"])["address"]
    verifications_dict.setdefault(row["fid"], set()).add(address)


In [62]:
verifications_dict

{1731: {'0x4cbc2e1c86c5ea7c5ea742a4e77696e4852840b5'},
 2183: {'0xb4fc9a7b8be45c14b6a7364667069544ea4cbde0'},
 2283: {'0xc5d52fff41386bf8ec591d97441b89053e222e74'},
 7438: {'0x09de8de37fd4befe77a45eda7e794a2bebbdc1b7'},
 9595: {'0x81947aadf64cd22ce96999e4c76efb0765839bef'},
 10636: {'0x202bdfdb2750df12520e7c83ad3097a7b9c82976'},
 13184: {'0xfe6b037f7029af049bed8885e87783352fcd61d4'},
 13220: {'0xcbc7d0ff51d37b60ba741bf566496bba53b5eea2'},
 8594: {'0x258d4c329b6dab96e9f01b89ab203d5d5108ce2b'},
 202699: {'0xe855f2fb565ed70dd14dc278a0834fa7814a62f0'},
 16914: {'0x8a29fc314308252429b53634387c446b496f7897',
  '0xeea2ee7a532e04d5489627401a45bd1ec9f69b02'},
 212073: {'0x328283d917f5d61ac73953bbf3810960eefc399d'},
 214570: {'0x17daf278f0d4e406b435ea74387f52d0b18c5c25',
  '0x51cb6dbbabf4f1ed325dbdc9bd448b317e204e4571122c39924e31c4887d4d56'},
 200530: {'0x4eec70d5c99acec3a3762f2bb13f4f0a7ca80a67'},
 234568: {'0x75c88d2149e00a36dddb997faf674ec9b57aeb73'},
 229621: {'0x9ab061f5f8f484328f195748d670

In [85]:
import time

def get_stamps_batch(address, max_retries=15):
    url = f"https://api.scorer.gitcoin.co/registry/stamps/{address}"
    headers = {"X-API-KEY": apikey, "accept": "application/json"}
    retries = 0

    while retries < max_retries:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            if len(response.json().get("items", [])) > 0:
                return True
            return False
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                retries += 1
                wait_time = 2 ** retries
                print(f"Rate limit reached. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            time.sleep(2 ** retries)
    
    return False

# final_result = {fid: None for fid in verifications_dict.keys()}
with open('final_result.json', 'r') as json_file:
    final_result = json.load(json_file)
    final_result = {int(key): value for key, value in final_result.items()}

for fid, addresses in verifications_dict.items():
    for address in addresses:
        if isinstance(final_result[fid], bool):
            continue
        if final_result[fid]:
            continue
        if Web3.is_address(address):
            if get_stamps_batch(address):
                final_result[fid] = True
                break
        final_result[fid] = False

Rate limit reached. Retrying in 2 seconds...
Rate limit reached. Retrying in 4 seconds...
Rate limit reached. Retrying in 8 seconds...
Rate limit reached. Retrying in 16 seconds...
Rate limit reached. Retrying in 32 seconds...
Rate limit reached. Retrying in 64 seconds...
Rate limit reached. Retrying in 128 seconds...
Rate limit reached. Retrying in 2 seconds...
Rate limit reached. Retrying in 4 seconds...
Rate limit reached. Retrying in 8 seconds...
Rate limit reached. Retrying in 16 seconds...
Rate limit reached. Retrying in 32 seconds...
Rate limit reached. Retrying in 64 seconds...
Rate limit reached. Retrying in 128 seconds...
Rate limit reached. Retrying in 256 seconds...
Rate limit reached. Retrying in 2 seconds...
Rate limit reached. Retrying in 4 seconds...
Rate limit reached. Retrying in 8 seconds...
Rate limit reached. Retrying in 16 seconds...
Rate limit reached. Retrying in 32 seconds...
Rate limit reached. Retrying in 64 seconds...
Rate limit reached. Retrying in 128 seco

In [77]:
# [1731, 2183, 2283, 13220, 8594, 214570, 200530, 229621]
# final_result

{1731: True,
 2183: True,
 2283: True,
 7438: False,
 9595: False,
 10636: False,
 13184: False,
 13220: True,
 8594: True,
 202699: False,
 16914: False,
 212073: False,
 214570: True,
 200530: True,
 234568: False,
 229621: True,
 234855: False,
 18529: False,
 240698: False,
 221475: True,
 240339: True,
 243573: True,
 234704: True,
 239294: False,
 246384: True,
 246871: True,
 248150: False,
 248035: False,
 265797: True,
 262906: False,
 269692: True,
 247542: False,
 245872: False,
 271815: False,
 272717: True,
 276128: False,
 16851: False,
 278838: False,
 279766: False,
 268934: False,
 281657: False,
 271381: False,
 286404: True,
 287128: False,
 288687: False,
 282129: False,
 238967: True,
 243825: False,
 296420: True,
 6158: False,
 245042: True,
 298334: False,
 292199: True,
 305490: False,
 306822: False,
 14345: True,
 309849: True,
 264970: False,
 310953: True,
 300175: False,
 287794: False,
 286476: False,
 286375: True,
 288788: True,
 243906: False,
 267965:

In [87]:
with open('final_result.json', 'w') as json_file:
    json.dump(final_result, json_file, indent=4)

In [86]:
final_result

{1731: True,
 2183: True,
 2283: True,
 7438: False,
 9595: False,
 10636: False,
 13184: False,
 13220: True,
 8594: True,
 202699: False,
 16914: False,
 212073: False,
 214570: True,
 200530: True,
 234568: False,
 229621: True,
 234855: False,
 18529: False,
 240698: False,
 221475: True,
 240339: True,
 243573: True,
 234704: True,
 239294: False,
 246384: True,
 246871: True,
 248150: False,
 248035: False,
 265797: True,
 262906: False,
 269692: True,
 247542: False,
 245872: False,
 271815: False,
 272717: True,
 276128: False,
 16851: False,
 278838: False,
 279766: False,
 268934: False,
 281657: False,
 271381: False,
 286404: True,
 287128: False,
 288687: False,
 282129: False,
 238967: True,
 243825: False,
 296420: True,
 6158: False,
 245042: True,
 298334: False,
 292199: True,
 305490: False,
 306822: False,
 14345: True,
 309849: True,
 264970: False,
 310953: True,
 300175: False,
 287794: False,
 286476: False,
 286375: True,
 288788: True,
 243906: False,
 267965:

In [97]:
truly_non_bots = [key for key, value in final_result.items() if value]
possibly_non_bots = [key for key, value in final_result.items() if not value]
len(truly_non_bots), len(possibly_non_bots)

(88, 429)

In [98]:
fnames_df.filter(pl.col('fid').is_in(truly_non_bots)).unique('fid')

created_at,updated_at,custody_address,expires_at,fid,deleted_at,fname
datetime[ns],datetime[ns],binary,datetime[ns],i64,datetime[ns],str
2024-03-30 01:57:36.386332,2024-06-08 05:59:05.605,"b""b\xb5\xf0\xda)\xd2\x94\xb3\xb3\x16\x95q1\xdd\x83\xcfG\xdb\x12i""",null,418270,null,"""jung8882"""
2024-02-28 20:45:29.414103,2024-06-08 01:57:25.086,"b""\xcd+\x06\x17\xb6\x09\xbc\x16\x9a>\x01w\xccE\xd8\xa7\x9b\x13\x09\xaa""",null,374893,null,"""vladdeth"""
2024-01-30 22:46:55.561473,2024-06-07 19:21:24.821,"b""\x87\x08zV\xb1\xd9I;\xf2\xf0\xc2(\x8e\xc6\x04\xee\x87\xf9h\xef""",null,245042,null,"""summercloud"""
2024-02-05 01:17:39.290474,2024-06-07 21:36:19.230,"b""\xf4\xb5R\x92\x88\xd9\xf5E\x8ecF|\xf2)+%\xd5\x03bF""",null,284274,null,"""zinisgood"""
2024-03-29 23:22:06.153043,2024-06-08 05:57:59.453,"b""\x7fp\x82\xa2?\xe1z\xcc\xc2\xf7>\x8a\xce[x\xe4]\xdfG{""",null,417936,null,"""busanlyk"""
…,…,…,…,…,…,…
2024-02-07 15:26:17.800160,2024-06-07 23:37:54.815,"b""$\xcf5gB(U\xb1\x83!\xd2\x0d\xf0\x0b.\x90\xd9\xe6\x81\x0b""",null,317735,null,"""roran82"""
2024-01-27 07:43:00.853941,2024-06-07 18:55:21.713,"b""\xba\x8a\xda\xe1\x1d\xeeU\xfd\x9d5\x82\x99EBG_\xc4T\xbb&""",null,234704,null,"""musicaux"""
2024-02-05 08:52:20.152075,2024-06-07 21:56:36.746,"b""\x10\xda\xe9\xf0\xbfE\xe3+2\xdd\xf2\xf5Z\x01\xdapc\x8e:m""",null,288788,null,"""leon5366"""


In [99]:
fnames_df.filter(pl.col('fid').is_in(possibly_non_bots)).unique('fid')

created_at,updated_at,custody_address,expires_at,fid,deleted_at,fname
datetime[ns],datetime[ns],binary,datetime[ns],i64,datetime[ns],str
2024-03-27 22:25:05.990688,2024-06-08 05:26:16.606,"b""\x08\xfa^,\xc4\x05\xbc\xd4K!\xb7\x893\x11\xee\xb8\x08S\x09\x13""",null,412917,null,"""wooning"""
2024-04-20 10:16:58.260369,2024-06-08 13:12:52.446,"b""\xd0w\xc5\xe7\xbcOe\x12\xd5q\x10\xbc\x88\xdd\xea\xa1\xc6\xb0*\x04""",null,493845,null,"""ayoboy"""
2024-01-31 05:37:49.002906,2024-06-07 19:23:24.650,"b""\xe9\x97\x0f&\x82$\x8e\x12y\x86\x02\xee\xb1\xf1\x1b\xf8\x80\x9e\xbe\x83""",null,245872,null,"""cognac"""
2024-03-22 15:21:16.116996,2024-06-08 04:06:20.647,"b""\xbel\x98\xba\xf3h\x80\xe2p\x9a\x8e\xbe\xfanf\xaf\xb1\x02Z\xc9""",null,402031,null,"""victorbueno"""
2024-02-05 14:34:26.368602,2024-06-07 22:15:26.045,"b"";\x10\x0a\x80\xd5\xc4\x0b\xe2\x9b\x1dai\xeb\xcb]\xae\x1d\xca\xca\xf0""",null,293198,null,"""ironhuntero"""
…,…,…,…,…,…,…
2024-03-06 08:31:48.759096,2024-06-08 02:29:06.998,"b""\xda\xf0J\xbaMlW\x1e51X\x8f\x9d&M\x8a\x85\xe3AJ""",null,380174,null,"""blancc"""
2024-02-05 05:53:48.254221,2024-06-07 21:43:14.666,"b""\x1f\xa3\x15\xb9\x94\xc6\x01\x0fkA\xfb1!CAd\xb8@U\xe1""",null,286490,null,"""smchoi"""
2024-04-25 03:27:22.946564,2024-06-08 13:52:16.525,"b""JWx\x90;\xa9\x9c\xe4\xd3\x9e\x9d\xd6\xbbe\x89\xa6\x15QV\xc3""",null,500336,null,"""wowhjj"""


In [ ]:
manual_inspection_non_bots = ["dankyhashpantz"]

In [101]:
scarbenign_bots = bots_posterior.filter(pl.col('posterior')>0.5)


In [102]:
scarbots_sybils = non_bots_posterior.filter(pl.col('posterior')<=0.5)
scarbots_sybils_verifications = verifications_df.filter(pl.col('fid').is_in(scarbots_sybils.select('fid')) & pl.col('deleted_at').is_not_null())
scarbots_sybils_verifications

created_at,updated_at,deleted_at,timestamp,fid,hash,claim,id
datetime[ns],datetime[ns],datetime[ns],datetime[ns],i64,binary,str,i64
2023-09-05 23:15:32.256357,2023-09-05 23:15:32.256357,2023-09-20 23:23:08.284,2023-09-03 21:32:35,7633,"b""\xd6%\x96@U\x11\xaeaI\xd4,^\x1d\xee\xf9\xa1\x11k\xf1}""","""{""address"": ""0xcd0b67a61e5e8f4616c19e421e929813b6d947df"", ""blockHash"": ""0x724cb51bf6ae72e83d67321a2ac83daf462dc41baa224ed8e1d674d66fcadacd"", ""ethSignature"": ""0x373b87b4e4c5ef1f1229f95d21325c19737bd1e631b14f352a788695d4f548a64bde182873b932f5b33b0cb76cb0d49a283699548afdea80a1c0fce44a070ce21c""}""",2392
2023-09-05 23:19:30.568741,2023-09-05 23:19:30.568741,2024-04-02 03:19:43,2023-05-05 00:41:29,12687,"b""\xabS\xceo\x01\xadg\xe6\xa3~\x94(\xbc\xdd\xf8;G@1\xbf""","""{""address"": ""0x4079042404ed1c5cdc780394c979feb99eae2bd2"", ""blockHash"": ""0x375de4aad0a7331c435c7b7814b43a518bdca3407c57927bbc372c05e9b1de66"", ""ethSignature"": ""0x0721ea183521f28131b1ec69de98bc331ac5861a5faf4939c2773505972eb2804e4916798860c0dd02c7f18b779506e6a2080bd6e84a4aa3211938679fd9176b1b""}""",4722
2023-09-05 23:20:20.734123,2023-09-05 23:20:20.734123,2024-02-26 08:00:17,2023-07-17 17:33:29,13666,"b""w\xdcca\xcc-\x85\xc7rn\x1f\x91\xa6Z\xc4\xa3Y\x0d]\xf3""","""{""address"": ""0x4967927f77311874f689719af48a774cea31196d"", ""blockHash"": ""0x4532b7b2719f94c8ae64a851a004cd6c2be0093d3b70a85fedd7383762cf23f9"", ""ethSignature"": ""0x12003d4e2288c4c9b422073f14acfda9e8cb0d08118b04acb1e9a48ee4c8c4dd7e60d3c3b7c12bceb8b28b7f03d8d8ecf3439e79b0cff498b3e418b92d6ec9091b""}""",5258
2023-09-05 23:20:21.311834,2023-09-05 23:20:21.311834,2023-11-25 04:33:46,2023-06-08 11:47:48,13674,"b""\xe5\x18Y\'\xd1\xda\x93\x85\x06W\xefP\xc1\xfc\xcew\xe8\xdf0""","""{""address"": ""0xa032af19029667fc7ad842d65d524e62a16cc4ca"", ""blockHash"": ""0x8187fd320a50e7dc5dab3bff366d3c849a8921067765d2a59d5adbc51c832c0e"", ""ethSignature"": ""0xf841198a4abb12b339a64dbd40263ce63a53710ca60689843990189d998194bf526fb60b4067a5addffe44ebb309f9cf0c7ca83ba039c28e3707aa53380b32d61b""}""",5265
2023-09-05 23:20:35.591922,2023-09-05 23:20:35.591922,2024-03-13 19:09:23,2023-06-15 16:09:04,13905,"b""\xd7X\x20-o>\x8e\xfcw\x9c\x8d\xde/\xbd\x9c\xa4\xa7\xf7\x83\xe5""","""{""address"": ""0x06e84ad48a328e2b7c4dfb172c4edfb9fba6bd9b"", ""blockHash"": ""0xb2893cdd20fbf6184eebd3c1051aeeea779b7d9077e1dbf4492ff8c39c8d3bbd"", ""ethSignature"": ""0xa37d0e5a6ef71a2c2c3514a22738c322940a5b3468914bc0dc4de88a60b4f2fd7afd04f9e4ba212d7181c980429ad4f28fe8109b41099c018a5735e7642589221b""}""",5386
…,…,…,…,…,…,…,…
2024-08-01 21:48:50.057606,2024-10-01 23:05:14.714,2024-10-01 23:05:13,2024-08-01 21:48:50,238622,"b""\x0f\x0c\x0c\xd6\xe56\xb1J\xd2N0\xb8/\xa5\xb7\xcd\x01z\x20=""","""{""address"": ""0x8224084767af2545c26f5dde0343809a781fb2ba"", ""blockHash"": ""0x9896bd23e801c88f16437f000eb8707c3f8dc65f362acae1fca4a4bf836a1a84"", ""ethSignature"": ""0x000000000000000000000000ca11bde05977b3631167028862be2a173976ca1100000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000280000000000000000000000000000000000000000000000000…",36383507
2024-06-21 06:06:53.410100,2024-10-01 23:05:14.714,2024-10-01 23:05:13,2024-06-21 06:06:33,238622,"b""\xea\xc1\x15\xd8'\xa0\xc5\x94\xeb\x1dl\xb0+f\x8b\x0b\xbf\x14""\xc5""","""{""address"": ""0x8224084767af2545c26f5dde0343809a781fb2ba"", ""blockHash"": ""0xb51e88588ef74a75cc37301a68a47559be1a9c4a2695cafbc39ec34f397a1257"", ""ethSignature"": ""0x0000000000000000000000000ba5ed0c6aa8c49038f819e587e2633c4a9f428a00000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000180000000000000000000000000000000000000000000000000…",36298979
2024-02-11 13:28:10.143163,2024-10-02 05:23:21.949,2024-10-02 05:23:21,2024-02-11 13:28:07,338699,"b""\xfa\xa6\xf4\x87\x05\xbb\x84A\xf6]\xb0\xf7Z\x84\xd0\xc5w\xf4F>""","""{""address"": ""0x99d46ced5c6baf476a94b71e2bedc064311c5168"", ""blockHash"": ""0x13813f23

In [103]:
verifications_dict = {}
for row in sybilbenign_bots_verifications.select(["fid", "claim"]).to_dicts():
    address = json.loads(row["claim"])["address"]
    verifications_dict.setdefault(row["fid"], set()).add(address)
verifications_dict

{1731: {'0x4cbc2e1c86c5ea7c5ea742a4e77696e4852840b5'},
 2183: {'0xb4fc9a7b8be45c14b6a7364667069544ea4cbde0'},
 2283: {'0xc5d52fff41386bf8ec591d97441b89053e222e74'},
 7438: {'0x09de8de37fd4befe77a45eda7e794a2bebbdc1b7'},
 9595: {'0x81947aadf64cd22ce96999e4c76efb0765839bef'},
 10636: {'0x202bdfdb2750df12520e7c83ad3097a7b9c82976'},
 13184: {'0xfe6b037f7029af049bed8885e87783352fcd61d4'},
 13220: {'0xcbc7d0ff51d37b60ba741bf566496bba53b5eea2'},
 8594: {'0x258d4c329b6dab96e9f01b89ab203d5d5108ce2b'},
 202699: {'0xe855f2fb565ed70dd14dc278a0834fa7814a62f0'},
 16914: {'0x8a29fc314308252429b53634387c446b496f7897',
  '0xeea2ee7a532e04d5489627401a45bd1ec9f69b02'},
 212073: {'0x328283d917f5d61ac73953bbf3810960eefc399d'},
 214570: {'0x17daf278f0d4e406b435ea74387f52d0b18c5c25',
  '0x51cb6dbbabf4f1ed325dbdc9bd448b317e204e4571122c39924e31c4887d4d56'},
 200530: {'0x4eec70d5c99acec3a3762f2bb13f4f0a7ca80a67'},
 234568: {'0x75c88d2149e00a36dddb997faf674ec9b57aeb73'},
 229621: {'0x9ab061f5f8f484328f195748d670

In [104]:
final_result_scarsybils_bots = {fid: None for fid in verifications_dict.keys()}
# with open('final_result_scarsybils_bots.json', 'r') as json_file:
#     final_result_scarsybils_bots = json.load(json_file)
#     final_result_scarsybils_bots = {int(key): value for key, value in final_result_scarsybils_bots.items()}

for fid, addresses in verifications_dict.items():
    for address in addresses:
        if isinstance(final_result_scarsybils_bots[fid], bool):
            continue
        if final_result_scarsybils_bots[fid]:
            continue
        if Web3.is_address(address):
            if get_stamps_batch(address):
                final_result_scarsybils_bots[fid] = True
                break
        final_result_scarsybils_bots[fid] = False

Rate limit reached. Retrying in 2 seconds...
Rate limit reached. Retrying in 4 seconds...
Rate limit reached. Retrying in 8 seconds...
Rate limit reached. Retrying in 16 seconds...
Rate limit reached. Retrying in 32 seconds...
Rate limit reached. Retrying in 64 seconds...
Rate limit reached. Retrying in 128 seconds...
Rate limit reached. Retrying in 256 seconds...
Rate limit reached. Retrying in 512 seconds...
Rate limit reached. Retrying in 2 seconds...
Rate limit reached. Retrying in 4 seconds...
Rate limit reached. Retrying in 8 seconds...
Rate limit reached. Retrying in 16 seconds...
Rate limit reached. Retrying in 32 seconds...
Rate limit reached. Retrying in 64 seconds...
Rate limit reached. Retrying in 128 seconds...
Rate limit reached. Retrying in 256 seconds...
Rate limit reached. Retrying in 2 seconds...
Rate limit reached. Retrying in 4 seconds...
Rate limit reached. Retrying in 8 seconds...
Rate limit reached. Retrying in 16 seconds...
Rate limit reached. Retrying in 32 sec

In [107]:
with open('final_result_scarsybils_bots.json', 'w') as json_file:
    json.dump(final_result_scarsybils_bots, json_file, indent=4)

In [108]:
len(final_result_scarsybils_bots)

517